In [ ]:
!pip install flask transformers huggingface_hub pyngrok
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

!ngrok config add-authtoken 2pOkYySnSPEfSzC8mmUuYHXRTr0_7eN6ZAq6htCGNcg8sJWU1

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,185 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyngrok import ngrok

# Set up a tunnel to the Flask app
public_url = ngrok.connect(5004)
print(f"Flask app is live at {public_url}")

Flask app is live at NgrokTunnel: "https://7756-34-125-148-182.ngrok-free.app" -> "http://localhost:5004"


In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

app = Flask(__name__)

#For drive model
drive_path = '/content/drive/MyDrive/gpt_neo2'
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(drive_path)
tokenizer = AutoTokenizer.from_pretrained(drive_path)

if device == "cuda":
    model = model.cuda()  # Move to GPU
else:
    model = model.cpu()   # Ensure it's on the CPU

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.json
        print(data)
        prompt = data.get("input", "")

        if not prompt:
            return jsonify({"error": "No prompt provided"}), 400

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        # suggestions = []
        # outputs = [model.generate(
        #     inputs["input_ids"],
        #     max_length=100,
        #     num_return_sequences=1,
        #     temperature=0.5,
        #     do_sample=True,
        #     top_p=0.9,
        #     pad_token_id=tokenizer.eos_token_id
        # ),
        # model.generate(
        #     inputs["input_ids"],
        #     max_length=100,
        #     num_return_sequences=1,
        #     temperature=0.6,
        #     top_k=100,
        #     do_sample=True,
        #     pad_token_id=tokenizer.eos_token_id
        # ),
        # model.generate(
        #     inputs["input_ids"],
        #     max_length=100,
        #     num_return_sequences=1,
        #     do_sample=True,
        #     top_k=100,
        #     temperature=0.8,
        #     pad_token_id=tokenizer.eos_token_id
        # )]
        # for output in outputs:
        #   for sequence in output:
        #     suggestions.append(tokenizer.decode(sequence, skip_special_tokens=True))

        # Generate multiple outputs manually
        suggestions = []
        for _ in range(3):  # Generate three suggestions
            output = model.generate(
                inputs.input_ids,
                max_length=len(prompt)+16,
                temperature=0.8,  # Control randomness
                top_k=50,         # Limit to top-k tokens
                do_sample=True,   # Enable sampling for diversity
            )
            decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
            formatted_output,_,_ = decoded_output[len(prompt):].rpartition('\n')
            suggestions.append(formatted_output)

        # Sort by length (as a proxy for "accuracy" or completion suitability)
        sorted_suggestions = sorted(set(suggestions), key=lambda x: len(x))  # Remove duplicates
        for suggestion in sorted_suggestions:
          print(suggestion)
        response = jsonify({"suggestions": sorted_suggestions})


        return response

    except Exception as e:
        print("Error during generation:", e)
        return jsonify({"error": str(e)}), 500

app.run(host='0.0.0.0', port=5004)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5004
 * Running on http://172.28.0.12:5004
INFO:werkzeug:Press CTRL+C to quit
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input': '-- '}
{'input': '-- '}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'input': '-- '}
{'input': '-- '}
{'input': '-- '}
{'input': '-- '}
{'input': '-- '}
{'input': '-- '}
{'input': '-- '}
{'input': '-- '}


...

{'input': '-- BCD to 7 segments\nlibrary      IEEE;\nuse      IEEE.STD_LOGIC_1164.ALL;\n\nentity  bcdto7seg is\n        port (\n            clk, reset: in  STD_LOGIC;\n            disp: '}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 in  STD_LOGIC_VECTOR (7 downto 0);
            bs: out STD_LOGIC_VECTOR (7 downto 0)
        );
 out STD_LOGIC_VECTOR(7 downto 0)
        );
end                                       -- bcdto7seg and the corresponding bcdto7seg component
 inout STD_LOGIC_VECTOR (7 downto 0);
            input_in: in STD_LOGIC_VECTOR (7 downto 0);
            output: out STD_LOGIC_VECTOR (7 downto 0)
        );
end entity;


INFO:werkzeug:127.0.0.1 - - [03/Dec/2024 17:17:39] "POST /generate HTTP/1.1" 200 -


 in  STD_LOGIC_VECTOR(7 downto 0);
            clk_clk: in  STD_LOGIC
        );
end       bcdto7seg;

architecture Behavioral of bcdto7seg is
begin
 in  STD_LOGIC_VECTOR (7 downto 0);
            segments: out STD_LOGIC_VECTOR (7 downto 0)
            );
end  bcdto7seg;

architecture Behavioral of bcdto7seg is
     out  STD_LOGIC_VECTOR(7 DOWNTO 0)
        );
end bcdto7seg;

architecture arch of bcdto7seg is
begin
    disp <= disp + 1;
end arch;

entity bcdto7seg_bcdto7seg is
